# Visites des pages contributions

Dans cette exploration, le but est de savoir, les conséquences sur le traffique qu'à eu la migration au nouveau format des pages contributions

Étape 1: Avoir le total des visites pour une contrib (générique + personnalisées) du 08/12/23 au 19/12/23

Étape 2: même chose du 12/01/24 au 23/01/24

## 1. Chargement des librairies

In [1]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)


contribs_1 = [
    "est-il-obligatoire-davoir-un-contrat-de-travail-ecrit-et-signe",
    "quelle-est-la-duree-du-preavis-en-cas-de-demission",
    "quelle-est-la-duree-de-preavis-en-cas-de-depart-a-la-retraite",
    "quelles-sont-les-consequences-du-non-respect-du-preavis-par-le-salarie-ou-lemployeur",
    "quand-le-salarie-a-t-il-droit-a-une-prime-danciennete-quel-est-son-montant",
    "a-quelles-indemnites-peut-pretendre-un-salarie-qui-part-a-la-retraite",
    "les-conges-pour-evenements-familiaux",
    "quelles-sont-les-primes-prevues-par-la-convention-collective"
]


contribs_2 = [
    "la-periode-dessai-peut-elle-etre-renouvelee",
    "quelle-est-la-duree-de-preavis-en-cas-de-licenciement"
]

# décommenter la période à tester

query_before_start = "2023-12-08"
query_before_end = "2023-12-19"

query_after_start = "2024-01-12"
query_after_end = "2024-01-23"

contribs = contribs_1


#query_before_start = "2023-10-21"
#query_before_end = "2023-12-06"

#query_after_start = "2023-12-08"
#query_after_end = "2024-01-23"

#contribs = contribs_2

In [2]:
def format_url(df: pd.DataFrame):
    df['url'] = df['url'].str.split('#').str[0].str.split('?').str[0]
    df = df[df["type"]=="visit_content"]
    df = df[~df['idVisit'].duplicated(keep=False)]
    return df

## 2. Récupération des queries sur elasticsearch

In [3]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY_BEFORE = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/contribution/" 
              }
            },
            {
              "range": {
               "logfile": {
                    "gte": query_before_start,
                    "lte": query_before_end
               }
              }
            }
          ]
        }
      }
}


QUERY_AFTER = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/contribution/" 
              }
            },
            {
              "range": {
               "logfile": {
                    "gte": query_after_start,
                    "lte": query_after_end
               }
              }
            }
          ]
        }
      }
}

connection with ElasticSearch established with success


In [4]:
logs_before = es_connector.execute_query(QUERY_BEFORE, "logs-new")
logs_before_formated = format_url(logs_before.copy())


  0%|          | 0/221719 [00:00<?, ?it/s]

In [5]:
logs_after = es_connector.execute_query(QUERY_AFTER, "logs-new")
logs_after_formated = format_url(logs_after.copy())

  0%|          | 0/275392 [00:00<?, ?it/s]

In [6]:
logs_after_formated

,timeSpent,timestamp,url,idVisit,lastActionDateTime,lastActionTimestamp,referrerName,referrerTypeName,referrerUrl,serverTimePretty,logfile,uvi,type,resultSelection,ccAction,query,recoSelection,recoType,idCc,cc,feedbackType,visited,suggestionPrefix,suggestionSelection,outil,outilAction,outilEvent
10,0.0,1705910384,https://code.travail.gouv.fr/contribution/1979-quelle-est-la-duree-du-preavis-en-cas-de-demission,56712205,2024-01-21 22:59:44,1705877984,Google,Search Engines,https://www.google.com/,23:59:44,2024-01-21,194787185,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,40.0,1705910239,https://code.travail.gouv.fr/contribution/3239-a-quelles-indemnites-peut-pretendre-un-salarie-qu...,56712126,2024-01-21 22:59:43,1705877983,www.cesu.urssaf.fr,Websites,https://www.cesu.urssaf.fr/,23:59:43,2024-01-21,-619771666,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,0.0,1705910354,https://code.travail.gouv.fr/contribution/quelles-sont-les-conditions-dindemnisation-pendant-le-...,56712191,2024-01-21 22:59:14,1705877954,Google,Search Engines,https://www.google.com/,23:59:14,2024-01-21,1236948838,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,1129.0,1705909190,https://code.travail.gouv.fr/contribution/1517-en-cas-darret-maladie-du-salarie-lemployeur-doit-...,56711277,2024-01-21 22:58:39,1705877919,Google,Search Engines,https://www.google.com/,23:58:39,2024-01-21,-697026188,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,0.0,1705910315,https://code.travail.gouv.fr/contribution/a-quelles-indemnites-peut-pretendre-un-salarie-qui-par...,56712177,2024-01-21 22:58:35,1705877915,Google,Search Engines,https://www.google.com/,23:58:35,2024-01-21,-2008797426,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275374,0.0,1705718296,https://code.travail.gouv.fr/contribution/176-les-conges-pour-evenements-familiaux,56582448,2024-01-19 17:38:16,1705685896,Google,Search Engines,https://www.google.com/,18:38:16,2024-01-19,661182364,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275386,0.0,1705718292,https://code.travail.gouv.fr/contribution/en-cas-darret-maladie-du-salarie-lemployeur-doit-il-as...,56582433,2024-01-19 17:38:12,1705685892,Google,Search Engines,https://www.google.com/,18:38:12,2024-01-19,1642484084,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275387,35.0,1705718230,https://code.travail.gouv.fr/contribution/1480-quelles-sont-les-conditions-dattribution-de-la-pr...,56582368,2024-01-19 17:37:44,1705685864,Google,Search Engines,https://www.google.com/,18:37:44,2024-01-19,-364757099,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275390,0.0,1705718246,https://code.travail.gouv.fr/contribution/3248-a-quelles-indemnites-peut-pretendre-un-salarie-qu...,56582392,2024-01-19 17:37:26,1705685846,Bing,Search Engines,https://www.bing.com/,18:37:26,2024-01-19,842903088,visit_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Compte pour chaque contributions du nombre de ligne

In [7]:
### Avant

In [8]:
print(f"AVANT", query_before_start, query_before_end)
print(f"Contrib | Visites")
print(f"-- | --")
for c in contribs:
    count = logs_before_formated['url'].apply(lambda x: c in x).sum()
    print(f"{c} | {count}")


AVANT 2023-12-08 2023-12-19
Contrib | Visites
-- | --
est-il-obligatoire-davoir-un-contrat-de-travail-ecrit-et-signe | 1785
quelle-est-la-duree-du-preavis-en-cas-de-demission | 10285
quelle-est-la-duree-de-preavis-en-cas-de-depart-a-la-retraite | 1853
quelles-sont-les-consequences-du-non-respect-du-preavis-par-le-salarie-ou-lemployeur | 1574
quand-le-salarie-a-t-il-droit-a-une-prime-danciennete-quel-est-son-montant | 9445
a-quelles-indemnites-peut-pretendre-un-salarie-qui-part-a-la-retraite | 7194
les-conges-pour-evenements-familiaux | 19744
quelles-sont-les-primes-prevues-par-la-convention-collective | 5840


In [9]:
### Après

In [10]:
print(f"APRES", query_after_start, query_after_end)
print(f"Contrib | Visites")
print(f"-- | --")
for c in contribs:
    count = logs_after_formated['url'].apply(lambda x: c in x).sum()
    print(f"{c} | {count}")

APRES 2024-01-12 2024-01-23
Contrib | Visites
-- | --
est-il-obligatoire-davoir-un-contrat-de-travail-ecrit-et-signe | 1533
quelle-est-la-duree-du-preavis-en-cas-de-demission | 11364
quelle-est-la-duree-de-preavis-en-cas-de-depart-a-la-retraite | 1430
quelles-sont-les-consequences-du-non-respect-du-preavis-par-le-salarie-ou-lemployeur | 1145
quand-le-salarie-a-t-il-droit-a-une-prime-danciennete-quel-est-son-montant | 7365
a-quelles-indemnites-peut-pretendre-un-salarie-qui-part-a-la-retraite | 5645
les-conges-pour-evenements-familiaux | 16097
quelles-sont-les-primes-prevues-par-la-convention-collective | 2844


In [11]:
filtered_logs = logs_before_formated[logs_before_formated['url'].str.contains('quelle-est-la-duree-de-preavis-en-cas-de-licenciement')]

filtered_logs.groupby(by=["url"]).agg({"idVisit": "nunique"}).sort_values(by="idVisit", ascending=False).head(1000)


,idVisit
url,
https://code.travail.gouv.fr/contribution/quelle-est-la-duree-de-preavis-en-cas-de-licenciement,656
https://code.travail.gouv.fr/contribution/650-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,179
https://code.travail.gouv.fr/contribution/2941-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,89
https://code.travail.gouv.fr/contribution/413-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,77
https://code.travail.gouv.fr/contribution/16-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,73
https://code.travail.gouv.fr/contribution/292-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,42
https://code.travail.gouv.fr/contribution/3209-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,39
https://code.travail.gouv.fr/contribution/2609-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,32
https://code.travail.gouv.fr/contribution/1090-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,24


In [12]:
filtered_logs = logs_after_formated[logs_after_formated['url'].str.contains('quelle-est-la-duree-de-preavis-en-cas-de-licenciement')]
filtered_logs.groupby(by=["url"]).agg({"idVisit": "nunique"}).sort_values(by="idVisit", ascending=False)

,idVisit
url,
https://code.travail.gouv.fr/contribution/quelle-est-la-duree-de-preavis-en-cas-de-licenciement,435
https://code.travail.gouv.fr/contribution/3248-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,392
https://code.travail.gouv.fr/contribution/16-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,70
https://code.travail.gouv.fr/contribution/2941-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,56
https://code.travail.gouv.fr/contribution/1501-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,20
https://code.travail.gouv.fr/contribution/2609-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,19
https://code.travail.gouv.fr/contribution/3043-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,18
https://code.travail.gouv.fr/contribution/843-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,18
https://code.travail.gouv.fr/contribution/3239-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,16
